In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Тестирование обученной LoRa модели

#### Инференс на одном изображении 

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline

# Пути к моделям
base_model = "stabilityai/stable-diffusion-xl-base-1.0"
lora_weights = "models/glam_test_task_data-05.safetensors"

# 1) Вариант для GPU (рекомендуется для fp16-инференса):
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float16,      # fp16 поддерживается на GPU
    use_safetensors=True,
).to("cuda")                        # перемещаем на GPU

# Подключаем LoRA
pipe.load_lora_weights(lora_weights)

# Генерация с явным тегом LoRA
prompt = "myface woman, professional portrait, soft lighting"
image = pipe(
    prompt,
    num_inference_steps=40,
    guidance_scale=7.5,
).images[0]
image.save("lora_result.png")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

#### Инференс на изображении без обученной LoRa и с обученной

In [17]:
import torch
from diffusers import StableDiffusionXLPipeline

base_model = "stabilityai/stable-diffusion-xl-base-1.0"
lora_weights = "models/glam_test_task_data-05.safetensors"

pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")

pipe.load_lora_weights(lora_weights)

# Латенты в float16!
generator = torch.Generator(device="cuda").manual_seed(45)

prompt_base = "woman, perfect details, high resolution, wearing augmented reality visor, standing in neon-lit market"
# Без LoRA-тега
image1 = pipe(
    prompt=prompt_base,
    generator=generator,
    num_inference_steps=40,
    guidance_scale=7.5,
).images[0]
image1.save("baseline.png")

# С LoRA-тегом
prompt_lora = "myface " + prompt_base
image2 = pipe(
    prompt=prompt_lora,
    generator=generator,
    num_inference_steps=40,
    guidance_scale=7.5,
).images[0]
image2.save("with_lora.png")

print("Готово — сгенерированы baseline.png и with_lora.png")

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Готово — сгенерированы baseline.png и with_lora.png
